In [9]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date, datetime, timedelta
from datetime import datetime

def soltrsupcol(M,n):
    z = n.copy()
    k = np.max(np.nonzero(n))
    for j in reversed(range(k + 1)):
        z[j] = z[j] / M[j,j]
        z[:j] = z[:j] - z[j] * M[: j, j]
    return z

def hh_vector(x):
    
    #u, rho = hh_vector(x)
    #Calcula u y rho tal que Q = I - rho u u^T
    #cumple Qx = \|x\|_2 e^1
    
    n = len(x)
    rho = 0
    u = x.copy()
    u[0] = 1.

    if n == 1:
        sigma = 0
    else:
        sigma = np.sum(x[1:]**2)

    if sigma>0 or x[0]<0:
        mu = np.sqrt(x[0]**2 + sigma)
        if x[0]<=0:
            gamma = x[0] - mu
        else:
            gamma = -sigma/(x[0] + mu)

        rho = 2*gamma**2/(gamma**2 + sigma)
        u = u/gamma
        u[0] = 1

    return u, rho

def qrhholder(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    p = min(m, n)

    for j in range(p):
        # I = j:, J = j:
        u, rho = hh_vector(R[j:, j])
        w = rho * u
        R[j:, j:] = R[j:, j:] - np.outer(w, u.T @ R[j:, j:])
        Q[:, j:] = Q[:, j:] - Q[:, j:] @ np.outer(w, u)

    return Q, R
    
def sol_cuadmin(A, b):
    m, n = A.shape

    #Asumimos que A tiene rango completo
    p = np.min([m,n])
    y_sol = np.zeros(n)
    # I = {1, ... , p} -> :p
    # J = {1, ... , p} \ I = {p+1, ... , n} -> p:

    Q, R = qrhholder(A)

    q = Q.T @ b

    y_sol[:p] = soltrsupcol(R[ :p, :p], q[:p])

    residuo = np.linalg.norm(q[p:])

    return y_sol, residuo

In [7]:
#Funcion que me da la Matriz de Datos de los precios de cierre en la ventana temporal indicada

# Formato fecha debe estas en mm-dd-yy

def Matriz_Precios(Tickers, start_date, end_date):

    #Convierto la fecha de clase str a clase date
    end_date =  datetime.strptime(end_date, '%Y-%m-%d')
    start_date = datetime.strptime(start_date, '%Y-%m-%d')

    #Busi = pd.date_range(start_date.date(), end_date.date(), freq='B')

    #Obtengo la cantidad de dias que tengo el dato de precio de las acciones
    N_columnas = len(yf.download('KO', start_date , end_date)['Close'])
    #Armo la matriz de precios, los cuales ire agregando
    A = np.zeros((len(Tickers), N_columnas))

    i = 0
    for company in Tickers:
        ith_company = yf.download(company, start_date , end_date)['Close']
        A[ i , :] = ith_company
        i += 1
    return(A)

Tickers = ['AAPL', 'MELI', 'MSFT', 'AMZN', 'KO', 'GOOGL', 'GS', 'DE', 'JPM', 'IBM']

Precios = Matriz_Precios(Tickers, '2010-06-01', '2011-06-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [55]:
#DMD.
X1 = Precios[ : ,0 : 17]
X2 = Precios[ : ,0+1 : 17+1]

def dmd(X1, X2, truncate=None):

    U2,Sig2,Vh2 = np.linalg.svd(X1, False) # SVD of input matrix

    r = len(Sig2) if truncate is None else truncate # rank truncation

    U = U2[:,:r]                    #
    Sig = np.diag(Sig2)[:r,:r]      #Trunco las matrices de la descomposicion SVD
    V = Vh2.conj().T[:,:r]          #

    Atil = U.conj().T @ X2 @ V @ np.linalg.inv(Sig) # build A tilde

    mu ,W = np.linalg.eig(Atil)

    Phi = X2 @ V @ np.linalg.inv(Sig) @ W # build DMD modes

    return mu, Phi, r

mu, Phi, r = dmd(X1,X2)

def x(t, r, mu, Phi):

    x = np.zeros(r)
    b = sol_cuadmin(Phi, X1[ : , 0])

    for k in range(r):
        x = x + Phi[: , k] * np.exp(np.log(mu[k] * t)) * b[0][k]

    return(np.real(x))

print(x(20,r,mu,Phi))
print(Precios[ : , 20])


[ 1353.66662472 -2921.0069777    332.78512133   -64.4742371
   948.47543522   -12.60709075  3012.74034687  3075.50989462
  2729.88483283  2237.94769654]
[  9.14892864  54.11999893  23.30999947   5.43050003  25.16500092
  11.36786842 133.75999451  55.43000031  37.06000137 119.58891296]


C:\Users\Pedro\AppData\Local\Temp\ipykernel_14172\2019823585.py:56: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[:, j:] = Q[:, j:] - Q[:, j:] @ np.outer(w, u)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_14172\2019823585.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  z[j] = z[j] / M[j,j]
C:\Users\Pedro\AppData\Local\Temp\ipykernel_14172\2019823585.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  z[:j] = z[:j] - z[j] * M[: j, j]
